# Data Cleaning / Wrangling

In [40]:
import os.path
import numpy as np
import networkx as nx
import pickle as pkl
import importlib
import sys
from collections import defaultdict, Counter

import controllability as ctrb
importlib.reload(ctrb)
import Towlson_group_code.data_io as myFunc
importlib.reload(myFunc)
import PREVENT_functions as prev_fct
importlib.reload(prev_fct)

print(sys.version)

PICKLE_PATH = '../../PREVENT_Study/pickles/'
# CONNECTOME_DATA_PATH = '../../PREVENT_Study/data/Individual_VolumeNormalized_dataframes/'
CONNECTOME_DATA_PATH = '../../PREVENT_Study/data/Individual_NONnormalized_dataframes/'
FIGURE_PATH = '../../PREVENT_Study/figures/'

3.8.12 (default, Mar  2 2022, 12:59:08) 
[Clang 13.0.0 (clang-1300.0.27.3)]


# Create Networks from Volume Normalized Connectomes
- [x] Calculate controllability values for each network
- [x] Record network metric values into graphs

In [58]:
def loadBrainCSV(time, stabilize=True):
    hc_data = {}
    p_data = {}
    # pklName = f'{time}_connectomes.pkl'
    pklName = f'{time}_nn_connectomes.pkl'
    bad_regions = []
    if os.path.exists(PICKLE_PATH+pklName):
        with open(PICKLE_PATH+pklName, 'rb') as f:
            hc_data = pkl.load(f)
            p_data = pkl.load(f)
    else:
        # regions to delete
        delete_regions =  ['Optic-Chiasm', '3rd-Ventricle', 'CSF', '4th-Ventricle', 'Left-vessel', 'Right-vessel', 'Left-Lateral-Ventricle', 'Left-Inf-Lat-Vent', 'Right-Inf-Lat-Vent', 'Right-Lateral-Ventricle', 'Brain-Stem', 'CC_Posterior', 'CC_Mid_Posterior', 'CC_Central', 'CC_Mid_Anterior', 'CC_Anterior', 'Left-Cerebral-White-Matter', 'Left-Cerebellum-White-Matter', 'Left-Cerebellum-Cortex', 'Right-Cerebral-White-Matter', 'Right-Cerebellum-White-Matter', 'Right-Cerebellum-Cortex', 'Left-VentralDC', 'Left-choroid-plexus', 'Right-VentralDC', 'Right-choroid-plexus', 'WM-hypointensities', 'ctx-lh-unknown', 'ctx-rh-unknown']
        delete_regions += ['Right-Pallidum', 'Left-Pallidum',
                           'ctx_lh_G_insular_short', 'ctx_rh_G_insular_short',
                           'ctx_lh_G_front_inf-Orbital', 'ctx_rh_G_front_inf-Orbital',
                           'ctx_rh_G_Ins_lg_and_S_cent_ins', 'ctx_lh_G_Ins_lg_and_S_cent_ins',
                           'ctx_rh_G_subcallosal', 'ctx_lh_G_subcallosal',
                           'ctx_lh_S_interm_prim-Jensen', 'ctx_rh_S_interm_prim-Jensen']

        # Load all pickle files from data source folder
        bad_files = 0
        n_population = 0
        hc_pop = 0
        tia_pop = 0
        stdOut = sys.stdout
        sys.stdout = open(f'../../PREVENT_Study/dump/{time}_error_log.txt', 'w')
        for root, dirs, files in os.walk(CONNECTOME_DATA_PATH):
            for file in (files):
                if not file.endswith('.pickle'):
                    continue
                if time not in file:
                    continue
                remove = ['371_BL_Control', '063_BL_Control', '174_BL_Control', '036_BL_TIA', '058_Y1_Control', '241_Y1_TIA', '253_Y1_TIA', '087_Y5_Control', '044_Y5_TIA', '198_Y5_TIA']
                skip = False
                for r in remove:
                    if r in file:
                        skip = True
                if skip:
                    continue

                with open(root+file, 'rb') as f:
                    cdf = pkl.load(f)
                # Clean up DF connectome
                adjMatrix = cdf.drop(delete_regions, axis=0)
                adjMatrix = adjMatrix.drop(delete_regions, axis=1)
                G = nx.from_pandas_adjacency(adjMatrix)
                if stabilize:
                    adjMat = adjMatrix.to_numpy()
                    if np.count_nonzero(adjMat) == 0:
                        print("No brain data #", patID)
                        continue
                    meanWeight = adjMat.sum() / np.count_nonzero(adjMat)
                    adjMatrix = adjMatrix.div(meanWeight)
                # --- Normalize G just to calculate controlllability
                G_norm = nx.from_pandas_adjacency(adjMatrix)

                # badGraph, G, badRegions = prev_fct.rank_nodes(G)
                # if badGraph:
                #     print("Can't rank node weights", file[:10])
                #     bad_files += 1
                #     for b in badRegions:
                #         bad_regions.append(b)
                #         print('\t*', b)
                #     continue
                    # cdf.to_excel('../PREVENT_study/data/bad/'+file[:len(file)-7]+'.xlsx')

                # Avg Controllability calc and ranking
                avgCtrbDict = ctrb.avg_control(G_norm)
                nx.set_node_attributes(G, avgCtrbDict, name='avgCtrb')
                # badGraph, G, badRegions = prev_fct.rank_nodes(G, attr='avgCtrb')
                # if badGraph:
                #     print(f"Could not rank avg ctrb for {file}")
                #     bad_files += 1
                #     for b in badRegions:
                #         bad_regions.append(b)
                #         print('\t*', b)
                #     continue

                # Modal controllability calc and ranking
                modalCtrbDict = ctrb.modal_control(G_norm)
                nx.set_node_attributes(G, modalCtrbDict, name='modCtrb')
                # badGraph, G, badRegions = prev_fct.rank_nodes(G, attr='modCtrb')
                # if badGraph:
                #     print(f"Could not rank mod ctrb for {file}")
                #     bad_files += 1
                #     for b in badRegions:
                #         bad_regions.append(b)
                #         print('\t*', b)
                #     continue

                info = file.split("_")
                patID = info[0]
                if info[2].lower() == 'tia':
                    # store in p_data as a networkX graph
                    p_data[patID] = G
                    tia_pop += 1
                else:
                    hc_data[patID] = G
                    hc_pop += 1
                n_population += 1
        with open(PICKLE_PATH+pklName, 'wb') as f:
            pkl.dump(hc_data, f)
            pkl.dump(p_data, f)
        print('\n-----------------------------------')
        print(f'Summary for {time} data set: ')
        print(f'There is a total of {n_population} good files.')
        print(f'   * {hc_pop} are Control')
        print(f'   * {tia_pop} are TIA')
        print(f'There are {bad_files} bad files that contained isolated nodes (hence not saved to cleaned data pickle).')
        print(f'In total the frequency of bad regions are: ')
        freq = sorted(Counter(bad_regions).items(), key=lambda x: x[1], reverse=True)
        print(*freq, sep="\n")

        sys.stdout = stdOut
    return hc_data, p_data, bad_regions

Once we have a pickle for each time frame, save them together in a single pickle.

In [ ]:
hc_bl_data, p_bl_data, bl_bad_regions = loadBrainCSV(time='bl')
hc_y1_data, p_y1_data, y1_bad_regions  = loadBrainCSV(time='Y1')
hc_y3_data, p_y3_data, y3_bad_regions  = loadBrainCSV(time='Y3')
hc_y5_data, p_y5_data, y5_bad_regions  = loadBrainCSV(time='Y5')

individual_data = {'HCbl': hc_bl_data, 'Pbl': p_bl_data, 'HCy1': hc_y1_data, 'Py1': p_y1_data,
                   'HCy3': hc_y3_data, 'Py3': p_y3_data, 'HCy5': hc_y5_data, 'Py5': p_y5_data}
myFunc.save_to_pickle(individual_data, PICKLE_PATH, 'Normalized_Connectomes.pkl')

# For FSLeyes visualization data prep

In [ ]:
with open('../../PREVENT_Study/Brain_Atlas/abbrev_to_label_mapping.txt') as f:
    lines = f.readlines()

mapping = {}
for line in lines:
    s = line.split(' ')
    if s[0] == '95':
        break
    lr = s[1].split('_')[-1]
    abbrv = s[3].rstrip('\n') + "." + lr
    # print(abbrv, s[1])
    mapping[abbrv] = s[1]

# Volume Data
- Store as pandas Dataframe pickles.
- Rename columns to correspond to our network's node names
- Everyone has different brain size. When you analyze region volume data you need to control for each person's varying brain volume.
Total Intracranial Volume (TIV) should be the same for each person regardless of aging effects. Therefore when we deal with
volume data, we should normalize each individual's nodal volume with TIV. There should be 1 TIV value per person and it should stay the same over the years.
In the {time}_volume.pkl files, these volumes have been normalized with the patient's TIV value.

There are more region names in the volume data sheets than the nodes we have in our brain networks.
Load each volume data and only select the node regions that matches the nodes we have in our brain network.

In [8]:
metadata, node_list = prev_fct.load_meta_data()
column_list = []
for n in node_list:
    if n[:3] == 'ctx':
        column_list.append(n[4:].replace("_and_", "&")+"_volume")
    elif 'Thalamus' in n:
        column_list.append(n+'-Proper')
    else:
        column_list.append(n)

158


- [X] Given the proper select of node volume data, normalize by the individual's TIV value.
- [X] Store dataframe as a pkl to use later.

In [29]:
# vol_df = myFunc.import_XLSX('../../PREVENT_Study/data/Region Volume data for participants/', 'Y5.xlsx')
# remove_columns = list(set(vol_df.columns) - set(column_list))
# vol_df = vol_df.drop(remove_columns, axis=1)
# rename_columns = {c: node_list[i]  for i ,c in enumerate(column_list)}
# vol_df.rename(columns=rename_columns, inplace=True)
t = 'y5'
bl_tiv = myFunc.import_XLSX('../../PREVENT_Study/data/', 'BL_TIV.xlsx')
vol_df = myFunc.load_from_pickle(PICKLE_PATH, f'{t}_volume.pkl')
key_error_list = []
for ri, row in vol_df.iterrows():
    try:
        p_tiv = bl_tiv.loc[ri][0]
        vol_df.loc[ri] = row/p_tiv
    except KeyError:
        key_error_list.append(ri)
print(len(key_error_list), key_error_list)
myFunc.save_to_pickle(vol_df, PICKLE_PATH, f'{t}_volume_tiv_normalized.pkl')

5 [55, 91, 175, 178, 181]
Saved to ../../PREVENT_Study/pickles/y5_volume_tiv_normalized.pkl.


# Non Volume Normalized Connectomes

Process non volume normalized connectomes just like the normalized ones.
Once we have a pickle for each time frame, save them together in a single pickle.
- [X] Removed ranking of nodes based on edge values.
- [X] Removed the 4 nodes that were giving issues in ranking before. Removed these 4 nodes even though we removed ranking
is because otherwise I need to create a new node_list and redo the volume data as well. Eventually this should be done,
but very quickly right now, just ignoring this in case we do decide to go back to ranking/volume normalized connectomes.

In [59]:
hc_bl_data, p_bl_data, bl_bad_regions = loadBrainCSV(time='BL')
hc_y1_data, p_y1_data, y1_bad_regions  = loadBrainCSV(time='Y1')
hc_y3_data, p_y3_data, y3_bad_regions  = loadBrainCSV(time='Y3')
hc_y5_data, p_y5_data, y5_bad_regions  = loadBrainCSV(time='Y5')

individual_data = {'HCbl': hc_bl_data, 'Pbl': p_bl_data, 'HCy1': hc_y1_data, 'Py1': p_y1_data,
                   'HCy3': hc_y3_data, 'Py3': p_y3_data, 'HCy5': hc_y5_data, 'Py5': p_y5_data}
myFunc.save_to_pickle(individual_data, PICKLE_PATH, 'Non_Normalized_Connectomes_2.pkl')

Saved to ../../PREVENT_Study/pickles/Non_Normalized_Connectomes_2.pkl.


Cross check result to look for isolated nodes. Manually check to see if these nodes are truly isolated (no edges).

In [60]:
from collections import Counter
bad_people = []
remove_list = {'HCbl': [], 'Pbl': [], 'HCy1': [], 'Py1': [],
               'HCy3': [], 'Py3': [], 'HCy5': [], 'Py5': []}
for patient_type in ['HC', 'P']:
    # fig, axs = plt.subplots(2, 2, figsize=(15, 15))
    if patient_type == 'HC':
        color = "tab:blue"
    if patient_type == 'P':
        color = 'tab:orange'
    for time in ['bl', 'y1', 'y3', 'y5']:
        data = individual_data[patient_type+time]
        print(time, patient_type, len(data))
        num_success = 0
        for pid, G in data.items():
            success, G, badRegions = prev_fct.rank_nodes(G, 'weight')
            if success == False:
                remove_list[patient_type+time].append(pid)
                # if len(badRegions) > 2:
                #     remove_list[patient_type+time].append(pid)
                # else:
                #     bad_people.append((patient_type+time, pid))
                    # print(pid, badRegions)
            else:
                num_success += 1
        print("\t success: ", num_success)

bad_regions = []
for a, b in bad_people:
    bad_person = individual_data[a][b]
    edges = bad_person.degree(weight='weight')
    for k, v in edges:
        if v == 0:
            bad_regions.append(k)
            avgC = nx.get_node_attributes(bad_person, "avgCtrb")[k]
            modC = nx.get_node_attributes(bad_person, "modCtrb")[k]
            # print(a, b, k, v, avgC, modC)

bl HC 178
	 success:  178
y1 HC 77
	 success:  77
y3 HC 36
	 success:  36
y5 HC 33
	 success:  33
bl P 231
	 success:  231
y1 P 100
	 success:  100
y3 P 17
	 success:  17
y5 P 34
	 success:  34


In [61]:
print(*Counter(bad_regions).most_common(), sep="\n")
amap = {'HC': 'Control', 'P': 'TIA'}
skip_list = []
for k, v in remove_list.items():
    print(k, len(v), v)
    if len(k) == 4:
        ptype = k[:2]
        time = k[2:]
    else:
        ptype = k[:1]
        time = k[1:]
    for l in v:
        skip_list.append(f'{l}_{time.upper()}_{amap[ptype]}')

print(skip_list)


HCbl 0 []
Pbl 0 []
HCy1 0 []
Py1 0 []
HCy3 0 []
Py3 0 []
HCy5 0 []
Py5 0 []
[]


In [67]:
import pandas as pd
G = individual_data['HCbl']['277']
node_df = pd.DataFrame(data={'node names': G.nodes})
node_df.to_excel('NodeList.xlsx', sheet_name='Sheet1', index=False, header=False)

Total cohort numbers by category.

In [36]:
metadata, node_list = prev_fct.load_meta_data()

print('P', len(metadata[(metadata['C/T'] == 'P')]))
print('HC', len(metadata[(metadata['C/T'] == 'HC')]))

P 218
HC 185
